In [1]:
from Base.MachiKoro_new._env import*

import numpy as np
import numba as nb
from scipy.stats.mstats import gmean, hmean
from numba import njit

In [2]:

@njit()
def bot_lv02(state, perData):
    validActions = getValidActions(state)
    arr_action = np.where(validActions == 1)[0]
    idx = np.random.randint(0, arr_action.shape[0])
    # print('qua')
    perData.append(state)
    return arr_action[idx], perData
per = []
per.append(np.zeros(P_LENGTH))
win, per = numba_main_2(bot_lv02, 1, per, 0)
# per

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'perData' of function 'check_run_under_njit'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "Base\MachiKoro_new\_env.py", line 880:
@njit()
def check_run_under_njit(agent, perData):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
c:\Users\Admin\Desktop\VIS_JOB\MAY_2023\ENV\Base\MachiKoro_new\_env.py:805: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'per_player' of function 'one_game_numba'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "B

In [19]:
a = np.where(getValidActions(per[32]))
a

(array([1, 2], dtype=int64),)

In [20]:
per[42][P_PHASE : P_PHASE + NUMBER_PHASE]

array([0., 0., 0., 0., 0., 0., 1.])

In [23]:
state = per[32]
b = getPredictState(state, 2)
b


(array([[8., 4., 2., ..., 0., 0., 0.],
        [2., 4., 2., ..., 0., 0., 0.],
        [7., 4., 2., ..., 0., 0., 0.],
        ...,
        [0., 4., 2., ..., 0., 0., 0.],
        [4., 4., 2., ..., 0., 0., 0.],
        [4., 4., 2., ..., 0., 0., 0.]]),
 array([0.02702703, 0.02702703, 0.08108108, 0.10810811, 0.13513514,
        0.16216216, 0.02702703, 0.13513514, 0.02702703, 0.08108108,
        0.08108108, 0.05405405, 0.02702703]))

In [15]:
state = per[32]
b = getPredictState(state, 2)
b.shape


(36, 146)

In [9]:
@njit()
def get_Prob_State2(arr_all_predict_state):
    new_arr_state = np.zeros((1, P_LENGTH))
    arr_count = np.array([0])
    len_new_arr_state = len(new_arr_state)
    test = 0
    while test < len(arr_all_predict_state):
        state_test = np.reshape(arr_all_predict_state[test], (1, P_LENGTH))
        new = True
        for j in range(len_new_arr_state):
            state_j = new_arr_state[j]
            if np.sum(state_test == state_j) == P_LENGTH:
                arr_count[j] += 1
                new = False
        if new:
            arr_count = np.append(arr_count, 1)
            new_arr_state = np.append(new_arr_state, state_test, axis= 0)
            len_new_arr_state += 1
        test += 1
    arr_prob = arr_count / np.sum(arr_count)
    return new_arr_state[1:],arr_prob[1:] 

c = get_Prob_State(b)
c   


(array([[8., 4., 2., ..., 0., 0., 0.],
        [2., 4., 2., ..., 0., 0., 0.],
        [7., 4., 2., ..., 0., 0., 0.],
        ...,
        [0., 4., 2., ..., 0., 0., 0.],
        [4., 4., 2., ..., 0., 0., 0.],
        [4., 4., 2., ..., 0., 0., 0.]]),
 array([0.02777778, 0.02777778, 0.08333333, 0.11111111, 0.13888889,
        0.16666667, 0.02777778, 0.13888889, 0.02777778, 0.08333333,
        0.08333333, 0.05555556, 0.02777778]))

In [52]:
@njit()
def getValidActions(player_state_origin):
    list_action_return = np.zeros(getActionSize(), dtype=np.float64)
    player_state = player_state_origin.copy()
    #  phase_env = np.where(player_state[P_PHASE : P_PHASE + NUMBER_PHASE] == 1)[0][0] + 1
    phase_env = -1
    if len(np.where(player_state[P_PHASE : P_PHASE + NUMBER_PHASE] == 1)[0]) != 0:
        phase_env = (
            np.where(player_state[P_PHASE : P_PHASE + NUMBER_PHASE] == 1)[0][0] + 1
        )
    else:
        print(list(player_state))
        raise Exception('toang phase getvalidAction')

    player_state_own = player_state[:ATTRIBUTE_PLAYER]
    """
        Quy ước phase: 
        phase1: chọn xúc sắc để đổ
        phase2: chọn đổ lại hay k
        phase3: chọn lấy tiền của ai
        phase4: chọn người để đổi
        phase5: chọn lá bài để đổi
        phase6: chọn lá bài muốn lấy
        phase7: chọn mua thẻ
    """
    if phase_env == 1:
        if player_state_own[-1] != 0:
            # chọn số xúc sắc để đổ: 1 ứng với 1 xúc sắc, 2 ứng với 2 xúc sắc
            list_action_return[np.array([1, 2])] = 1
        else:
            list_action_return[1] = 1

    elif phase_env == 2:
        # chọn đổ lại hay k, 0 là ko, 1 là đổ 1, 2 là đổ 2
        list_action_return[np.array([0, 1, 2])] = 1

    elif phase_env == 3:
        # 3, 4, 5 lần lượt là lấy tiền của người ở vị trí 1,2,3 sau mình
        all_player_coin = np.array(
            [
                player_state[ATTRIBUTE_PLAYER],
                player_state[ATTRIBUTE_PLAYER * 2],
                player_state[ATTRIBUTE_PLAYER * 3],
            ]
        )
        id_can_stole = np.where(all_player_coin > 0)[0]
        list_action_return[id_can_stole + 3] = 1

    elif phase_env == 4:
        # 6, 7, 8 lần lượt là chọn đổi thẻ với người ở vị trí 1,2,3 sau mình, 9 là ko đổi với ai
        list_action_return[np.array([6, 7, 8, 9])] = 1

    elif phase_env == 5:
        # duyệt trong các thẻ đang có, có thẻ nào đổi được thì đưa vào list_action range(10, 22)
        list_action = (
            np.where(player_state_own[1 : 1 + NUMBER_TYPE_NORMAL_CARD] > 0)[0] + 10
        )
        list_action_return[list_action] = 1
    elif phase_env == 6:
        # duyệt trong các thẻ của người chơi mình muốn đổi range(22-34)
        player_picked = np.where(
            player_state[P_PICKED_PLAYER : P_PICKED_PLAYER + NUMBER_PLAYER]
        )[0][0]
        player_picked_card = player_state[
            ATTRIBUTE_PLAYER * player_picked : ATTRIBUTE_PLAYER * (player_picked + 1)
        ][1 : 1 + NUMBER_TYPE_NORMAL_CARD]
        list_action_return[np.where(player_picked_card > 0)[0] + 22] = 1

    elif phase_env == 7:
        # chọn mua thẻ, hành động trải từ 34-53 với 53 là hành động bỏ qua ko mua thêm
        list_action_return[53] = 1
        p_coin = player_state[0]
        card_board = player_state[P_NORMAL_CARD:P_CARD_BUY_IN_TURN]
        card_bought = player_state[P_CARD_BUY_IN_TURN:P_CARD_SELL]
        if p_coin > 0:
            if card_board[0] > 0 and card_bought[0] == 0:
                # mua thẻ lúa mì
                list_action_return[34] = 1
        if p_coin > 0:
            if card_board[1] > 0 and card_bought[1] == 0:
                # mua thẻ nông trại
                list_action_return[35] = 1
        if p_coin > 0:
            if card_board[2] > 0 and card_bought[2] == 0:
                # mua thẻ tiệm bánh
                list_action_return[36] = 1
        if p_coin > 1:
            if card_board[3] > 0 and card_bought[3] == 0:
                # mua thẻ quán cà phê
                list_action_return[37] = 1
        if p_coin > 1:
            if card_board[4] > 0 and card_bought[4] == 0:
                # mua thẻ cửa hàng tiện lợi
                list_action_return[38] = 1
        if p_coin > 2:
            if card_board[5] > 0 and card_bought[5] == 0:
                # mua thẻ rừng
                list_action_return[39] = 1
        if p_coin > 4:
            if card_board[6] > 0 and card_bought[6] == 0:
                # mua thẻ nhà máy pho mát
                list_action_return[40] = 1
        if p_coin > 2:
            if card_board[7] > 0 and card_bought[7] == 0:
                # mua thẻ nhà máy nội thất
                list_action_return[41] = 1
        if p_coin > 5:
            if card_board[8] > 0 and card_bought[8] == 0:
                # mua thẻ mỏ quặng
                list_action_return[42] = 1
        if p_coin > 2:
            if card_board[9] > 0 and card_bought[9] == 0:
                # mua thẻ quán ăn gia đình
                list_action_return[43] = 1
        if p_coin > 2:
            if card_board[10] > 0 and card_bought[10] == 0:
                # mua thẻ vườn táo
                list_action_return[44] = 1
        if p_coin > 1:
            if card_board[11] > 0 and card_bought[11] == 0:
                # mua thẻ chợ trái cây
                list_action_return[45] = 1

        if p_coin > 5:
            if player_state_own[13] == 0:
                # mua thẻ sân vận động
                list_action_return[46] = 1
        if p_coin > 6:
            if player_state_own[14] == 0:
                # mua thẻ đài truyền hình
                list_action_return[47] = 1
        if p_coin > 7:
            if player_state_own[15] == 0:
                # mua thẻ trung tâm thương mại
                list_action_return[48] = 1

        if p_coin > 3:
            if player_state_own[-1] == 0:
                # mua thẻ 22 coin
                list_action_return[52] = 1
        if p_coin > 9:
            if player_state_own[-2] == 0:
                # mua thẻ 16 coin
                list_action_return[51] = 1
        if p_coin > 15:
            if player_state_own[-3] == 0:
                # mua thẻ 10 coin
                list_action_return[50] = 1
        if p_coin > 21:
            if player_state_own[-4] == 0:
                # mua thẻ 4 coin
                list_action_return[49] = 1

    return list_action_return


In [57]:
count = 0
for state in per[1:]:
    count += 1
    actions = np.where(getValidActions(state))[0]
    for action in actions:
        b = getPredictState(state, action)
        for i in range(len(b)):
            for j in range(i+1, len(b)):
                if np.sum(b[j] == b[i]) == 146:
                    print('trùng', i, j, action, count)

trùng 2 7 2 32
trùng 2 12 2 32
trùng 3 8 2 32
trùng 3 13 2 32
trùng 3 18 2 32
trùng 4 9 2 32
trùng 4 14 2 32
trùng 4 19 2 32
trùng 4 24 2 32
trùng 5 10 2 32
trùng 5 15 2 32
trùng 5 20 2 32
trùng 5 25 2 32
trùng 5 30 2 32
trùng 7 12 2 32
trùng 8 13 2 32
trùng 8 18 2 32
trùng 9 14 2 32
trùng 9 19 2 32
trùng 9 24 2 32
trùng 10 15 2 32
trùng 10 20 2 32
trùng 10 25 2 32
trùng 10 30 2 32
trùng 11 16 2 32
trùng 11 21 2 32
trùng 11 26 2 32
trùng 11 31 2 32
trùng 13 18 2 32
trùng 14 19 2 32
trùng 14 24 2 32
trùng 15 20 2 32
trùng 15 25 2 32
trùng 15 30 2 32
trùng 16 21 2 32
trùng 16 26 2 32
trùng 16 31 2 32
trùng 19 24 2 32
trùng 20 25 2 32
trùng 20 30 2 32
trùng 21 26 2 32
trùng 21 31 2 32
trùng 22 27 2 32
trùng 22 32 2 32
trùng 23 28 2 32
trùng 23 33 2 32
trùng 25 30 2 32
trùng 26 31 2 32
trùng 27 32 2 32
trùng 28 33 2 32
trùng 29 34 2 32
trùng 2 7 2 35
trùng 2 12 2 35
trùng 3 8 2 35
trùng 3 13 2 35
trùng 3 18 2 35
trùng 4 9 2 35
trùng 4 14 2 35
trùng 5 10 2 35
trùng 5 15 2 35
trùng 5 20 2 35

In [45]:
getValidActions(new[0])

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [46]:
new[0][P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER]

array([0., 1., 0., 0.])

In [9]:
P_CARD_SELL + 12 - P_PLAYER_CONTINUE

0

In [34]:
@njit()
def getPseudoAction(state, per_file):
    real_action = getValidActions(state)

    list_action_return = np.zeros(len(per_file[0]))
    list_action_return[:getActionSize()] = real_action

    for id in range(len(per_file[1])):
        action_real = np.where(per_file[1][id])[0][0]
        check = True
        for id_c in range(len(per_file[1][id])):
            if np.sum(state*per_file[2][id][id_c]) <= per_file[3][id][id_c]:
                check = False
                break
        if check and real_action[action_real] == 1:
            list_action_return[getActionSize() + id] = 1
        
    
    return list_action_return


@njit()
def agent_Hieu(state, per):
    bias = per[0][0][0]
    arr_actions = getPseudoAction(state, per)
    action = np.argmax(bias*arr_actions)
    if action == len(bias) - 2:
        action = 1
    elif action == len(bias) - 1:
        action = 4

    return action, per

from numba.typed import List, Dict

@njit()
def create_perx():
    perx = Dict()

    temp = List([np.zeros((1,500))])
    temp.pop(0)
    perx[0] = temp.copy()       #bias
    perx[1] = temp.copy()       #action real
    perx[2] = temp.copy()       #arr operator (+,-, constant)
    perx[3] = temp.copy()       #value compare
    perx[4] = temp.copy()


    return perx

per = create_perx()
per[0].append(np.random.rand(1,500))

arr_action_pseudo = np.zeros((1,500))
state_condition_operator = np.zeros((2,500))
value_compare = np.zeros((2,1))


arr_action_pseudo[0][1] = 1
state_condition_operator[0][17] = 1
state_condition_operator[1][6] = 1
value_compare[0] = 5
value_compare[1] = 3

per[1].append(arr_action_pseudo)
per[2].append(state_condition_operator)
per[3].append(value_compare)

arr_action_pseudo = np.zeros((1,500))
state_condition_operator = np.zeros((2,500))
value_compare = np.zeros((2,1))

arr_action_pseudo[0][4] = 1
state_condition_operator[0][3] = 1
state_condition_operator[0][5] = -1

state_condition_operator[1][4] = 1
state_condition_operator[1][2] = 1

value_compare[0] = 0
value_compare[1] = 7

per[1].append(arr_action_pseudo)
per[2].append(state_condition_operator)
per[3].append(value_compare)




In [4]:
import numpy as np
from numba import njit

@njit()
def test():
    a = np.zeros((2,15))
    a
    b = np.zeros(15)
    print(b)
    c = b.reshape(1,15)
    a = np.append(a,c, axis=0)
    return a
test()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [36]:
win, per = numba_main_2(agent_Hieu, 1000, per, 0)



ValueError: cannot assign slice from input of different size

In [6]:
b = np.random.randint(1,5,(2,3))
b

array([[1, 2, 3],
       [2, 1, 3]])

In [7]:
a*b

array([[4, 6, 3],
       [8, 2, 9]])

In [9]:
c = np.random.randint(1,5,(1,3))
c

array([[1, 1, 2]])

In [10]:
b*c

array([[1, 2, 6],
       [2, 1, 6]])

In [1]:
-3%4

1